In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the images
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Reshape the images to be (28, 28, 1) for CNNs (if you use CNNs later)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

2024-09-12 15:23:25.488625: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-12 15:23:25.489894: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-12 15:23:25.507879: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-12 15:23:25.507897: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-12 15:23:25.508405: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

def build_model(learning_rate=0.001, units=64, activation='relu'):
    model = Sequential([
        Flatten(input_shape=(28, 28, 1)),
        Dense(units, activation=activation),
        Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [3]:
from keras_tuner import GridSearch

def model_builder(hp):
    model = Sequential([
        Flatten(input_shape=(28, 28, 1)),
        Dense(units=hp.Int('units', min_value=32, max_value=128, step=32), 
              activation=hp.Choice('activation', values=['relu', 'tanh'])),
        Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

tuner = GridSearch(
    model_builder,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='mnist_grid_search'
)

tuner.search(x_train, y_train, epochs=5, validation_split=0.2)
best_model = tuner.get_best_models(num_models=1)[0]

Trial 10 Complete [00h 00m 05s]
val_accuracy: 0.9615833163261414

Best val_accuracy So Far: 0.9627500176429749
Total elapsed time: 00h 00m 51s


In [4]:
from keras_tuner import RandomSearch

def model_builder_random(hp):
    model = Sequential([
        Flatten(input_shape=(28, 28, 1)),
        Dense(units=hp.Int('units', min_value=32, max_value=128, step=32), activation=hp.Choice('activation', values=['relu', 'tanh'])),
        Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

tuner_random = RandomSearch(
    model_builder_random,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir_random',
    project_name='mnist_random_search'
)

tuner_random.search(x_train, y_train, epochs=5, validation_split=0.2)
best_model_random = tuner_random.get_best_models(num_models=1)[0]

Trial 10 Complete [00h 00m 12s]
val_accuracy: 0.9559166431427002

Best val_accuracy So Far: 0.9641666412353516
Total elapsed time: 00h 01m 18s


In [5]:
grid_search_results = best_model.evaluate(x_test, y_test)
print(f"Grid Search - Test Loss: {grid_search_results[0]}, Test Accuracy: {grid_search_results[1]}")

random_search_results = best_model_random.evaluate(x_test, y_test)
print(f"Random Search - Test Loss: {random_search_results[0]}, Test Accuracy: {random_search_results[1]}")

313/313 [==============================] - 0s 425us/step - loss: 0.1260 - accuracy: 0.9631
Grid Search - Test Loss: 0.12599678337574005, Test Accuracy: 0.963100016117096
313/313 [==============================] - 0s 397us/step - loss: 0.1156 - accuracy: 0.9659
Random Search - Test Loss: 0.11559081822633743, Test Accuracy: 0.9659000039100647
